In [2]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from train_cnn import EEGDataModule, EEGNetAutoencoder, EEGNetAutoencoderModel
import torch

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: billchen0011. Use `wandb login --relogin` to force relogin


In [3]:
import sys
sys.path.append("..")
from util import load_split_ids
train_ids, val_ids, test_ids = load_split_ids(Path("/home/bc299/icare/artifacts"))

In [2]:
path_to_data = Path("/media/hdd1/i-care/ten-seconds")

NameError: name 'Path' is not defined

In [76]:
dm = EEGDataModule(train_ids, val_ids, path_to_data)
model = EEGNetAutoencoder()

In [72]:
dm.setup()
train_sample = next(iter(dm.train_dataloader()))
val_sample = next(iter(dm.val_dataloader()))

In [42]:
train_sample.shape, val_sample.shape

(torch.Size([32, 1, 19, 1000]), torch.Size([32, 1000, 19]))

In [43]:
import torch.nn as nn
train_sample = train_sample.unsqueeze(1).permute(0, 1, 3, 2)

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 5 is not equal to len(dims) = 4

In [47]:
temporal = nn.Conv2d(1, 8, kernel_size=[19, 64], stride=1, bias=False, padding="same")
bn1 = nn.BatchNorm2d(8)
spatial = nn.Conv2d(8, 8*2, kernel_size=[19, 1], bias=False, groups=8)
bn2 = nn.BatchNorm2d(16)
elu = nn.ELU(True)
sepconv1 = nn.Conv2d(16, 16, kernel_size=[1, 16], padding="same", groups=16, bias=False)
sepconv2 = nn.Conv2d(16, 16, kernel_size=[1, 1], padding="same", groups=1, bias=False)
bn3 = nn.BatchNorm2d(16)
pool = nn.AvgPool2d([1, 5], stride=[1, 5], padding=0)

encoded = pool(elu(bn3(sepconv2(sepconv1(pool(elu(bn2(spatial(bn1(temporal(train_sample)))))))))))

In [67]:
import torch.nn.functional as F

upsamp = nn.Upsample(scale_factor=(1, 5), mode="nearest")
desep1 = nn.ConvTranspose2d(16, 16, kernel_size=[1, 16], groups=16, bias=False)
desep2 = nn.ConvTranspose2d(16, 16, kernel_size=[1, 1], groups=1, bias=False)
dbn1 = nn.BatchNorm2d(16)
de_spatial = nn.ConvTranspose2d(16, 8, kernel_size=[19, 1], groups=8, bias=False)
de_temporal = nn.ConvTranspose2d(8, 1, kernel_size=[1, 64], stride=1, bias=False)

decoded = de_temporal(de_spatial(upsamp(elu(dbn1(desep2(desep1(upsamp(encoded))[:, :, :, :200]))))))[:, :, :, :1000]

In [68]:
decoded.shape

torch.Size([32, 1, 19, 1000])

In [77]:
with torch.no_grad():
    model.eval()
    model(train_sample)

torch.Size([32, 1, 1000, 19])


In [10]:
from pathlib import Path
path_to_data = Path("/media/hdd1/i-care/ten-seconds/0284/0284_004_008_EEG.npy")
import numpy as np
np.max(np.load(path_to_data))

21.94972036340321

In [3]:
import torch
from torchviz import make_dot

model = EEGNetAutoencoderModel().cuda()
x = torch.randn(32, 1, 19, 1000).to(next(model.parameters()).device)
y = model(x)
make_dot(y, params=dict(model.named_parameters())).render("model", format="png")

/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1003.)
  return F.conv2d(input, weight, bias, self.stride,


'model.png'